## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-04-46-17 +0000,nyt,Poland Says It Shot Down Russian Drones That E...,https://www.nytimes.com/2025/09/10/world/europ...
1,2025-09-10-04-37-09 +0000,nyt,"Andrew Huse, Cultural Historian of the Cuban S...",https://www.nytimes.com/2025/09/09/dining/andr...
2,2025-09-10-04-32-15 +0000,wapo,Poland says it shot down Russian drones that v...,https://www.washingtonpost.com/world/2025/09/1...
3,2025-09-10-04-30-05 +0000,nyt,Explosive Protests in Nepal,https://www.nytimes.com/2025/09/10/briefing/ne...
4,2025-09-10-04-25-00 +0000,wsj,Poland Shoots Down Drones Deep Inside NATO Mem...,https://www.wsj.com/world/europe/poland-shoots...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2382/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
80,trump,54
136,new,19
231,israel,18
16,nepal,18
15,protests,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
107,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,136
33,2025-09-10-02-06-16 +0000,bbc,Trump says he's 'not thrilled' about Israel's ...,https://www.bbc.com/news/articles/ced58zywdwno...,124
324,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,113
69,2025-09-10-00-03-37 +0000,nypost,Trump asks EU to levy up to 100% tariffs on Ch...,https://nypost.com/2025/09/09/world-news/trump...,104
25,2025-09-10-02-45-20 +0000,nypost,Court rules that Lisa Cook can keep job as Fed...,https://nypost.com/2025/09/09/us-news/court-ru...,104


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
107,136,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...
196,79,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
219,77,2025-09-09-17-00-25 +0000,nyt,Protests Against Nepal’s Social Media Ban Show...,https://www.nytimes.com/2025/09/09/world/asia/...
45,61,2025-09-10-01-32-56 +0000,nypost,Iryna Zarutska’s family speaks out after Ukrai...,https://nypost.com/2025/09/09/us-news/iryna-za...
60,51,2025-09-10-00-36-34 +0000,nypost,Poland’s airspace ‘repeatedly violated’ by Rus...,https://nypost.com/2025/09/09/world-news/ukrai...
324,50,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...
206,42,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...
151,42,2025-09-09-20-54-54 +0000,nypost,Wife of late Sen. John McCain facing calls to ...,https://nypost.com/2025/09/09/us-news/cindy-mc...
23,41,2025-09-10-02-56-27 +0000,wapo,Judge temporarily halts firing of Fed governor...,https://www.washingtonpost.com/business/2025/0...
82,41,2025-09-09-23-20-31 +0000,nypost,Deranged Louisiana woman mows down coworker wi...,https://nypost.com/2025/09/09/us-news/louisian...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
